In [1]:
%cd ..


c:\Users\Francisco\Downloads\Input Data - Efficio Tech Task


c:\Users\Francisco\Downloads\Input Data - Efficio Tech Task\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import warnings
import pandas as pd
import ipywidgets as widgets

In [3]:
saving_plans = pd.read_csv("src/data/savingsplanrates.csv").drop(
    [
        "purchaseoption",
        "plantype",
        "sku",
        "ratecode",
        "unit",
        "contractlength",
        "rate_type",
        "discountedsku",
    ],
    axis=1,
)
saving_plans = saving_plans.query(
    "servicecode=='AmazonEC2' and operation in ['RunInstances','RunInstances:0002', 'Hourly']"
).drop("servicecode", axis=1)
saving_plans

,usagetype,operation,currency,rate,od_rate,unit_savings,perc_savings
10,EUC1-BoxUsage:c5.xlarge,RunInstances:0002,USD,0.2950,0.3780,0.0830,0.219577
16,BoxUsage:c5.xlarge,RunInstances,USD,0.0820,0.1700,0.0880,0.517647
20,BoxUsage:m5.xlarge,RunInstances,USD,0.0970,0.1920,0.0950,0.494792
25,BoxUsage:p3.2xlarge,RunInstances:0002,USD,2.1630,3.4280,1.2650,0.369020
26,BoxUsage:t2.micro,RunInstances:0002,USD,0.0104,0.0162,0.0058,0.358025
...,...,...,...,...,...,...,...
283,BoxUsage:m5a.large,RunInstances:0002,USD,0.1350,0.1780,0.0430,0.241573
285,BoxUsage:t3.large,RunInstances:0002,USD,0.0689,0.1108,0.0419,0.378159
293,USW1-BoxUsage:m5a.large,RunInstances:0002,USD,0.1510,0.1930,0.0420,0.217617
303,USW2-BoxUsage:r5a.large,RunInstances,USD,0.0590,0.1130,0.0540,0.477876


In [4]:
curr = pd.read_csv("src/data/currfile.csv", sep="|").drop(["lineitem/lineitemdescription"], axis=1)


curr.columns = [
    "operation",
    "usagetype",
    "timeinterval",
    "productcode",
    "usageamount",
    "ondemandrate",
]

In [ ]:
curr_ec2 = curr.query("operation in ['RunInstances','RunInstances:0002', 'Hourly']")
curr_ec2[["start_time", "end_time"]] = curr_ec2.timeinterval.str.split("/", expand=True)
curr_ec2["start_time"] = pd.to_datetime(curr_ec2.start_time, utc=True)
curr_ec2["end_time"] = pd.to_datetime(curr_ec2.end_time, utc=True)

In [6]:
default_start_date = pd.to_datetime("2023-1-1", utc=True)
default_end_date = pd.to_datetime("2023-1-14", utc=True)

commitment_amount = widgets.IntSlider(value=3, min=0, max=50, step=1, description="commitment")
start_date_picker = widgets.DatetimePicker(description="Start Date", value=default_start_date)
end_date_picker = widgets.DatetimePicker(description="End Date", value=default_end_date)

In [7]:
from src.utils.utils import get_within_and_outside_budget_expenses

warnings.filterwarnings("ignore", category=DeprecationWarning)



def render_choice(start_date_pck, end_date_pck, commitment_amount):
    start_date = pd.to_datetime(start_date_pck, utc=True)
    end_date = pd.to_datetime(end_date_pck, utc=True)
    usage_df = curr_ec2.query("start_time >= @start_date and end_time <= @end_date")

    ec2_calculated_spenditure = (
        usage_df.merge(saving_plans, on=["operation", "usagetype"], how="left")
        .sort_values(["end_time", "perc_savings"], ascending=[True, False])
        .assign(discount_price_cumulative=lambda df: (df.rate * df.usageamount).cumsum())
        .assign(haswithin_dicount_budget=lambda df: df.discount_price_cumulative <= commitment_amount)
    )

    within_outside_budget = ec2_calculated_spenditure.groupby("start_time").apply(
        get_within_and_outside_budget_expenses, commitment_amount=commitment_amount
    )

    all_possible_hours = (end_date - start_date).total_seconds() / 3600
    total_commited_budget_in_all_hours = commitment_amount * all_possible_hours
    with_discount_spent = within_outside_budget.within_budget.sum()
    without_discount_spent = within_outside_budget.outside_budget.sum()
    equivalent_without_discount = within_outside_budget.price_without_discount.sum()

    used_commited_budget_perc = with_discount_spent * 100 / total_commited_budget_in_all_hours
    print(f"Used Committed Budget: {used_commited_budget_perc:.0f}%")
    print(f"Extra On-Demand Cost: ${without_discount_spent:.2f}")
    print(f"Total Cost: ${total_commited_budget_in_all_hours + without_discount_spent:.2f}")
    print(f"Cost Without Discount: ${equivalent_without_discount:.2f}")
    savings = 100 * (1 - (total_commited_budget_in_all_hours + without_discount_spent) / equivalent_without_discount)
    print(f"Savings: {savings:.1f}%")

In [8]:
widgets.interact(
    render_choice, start_date_pck=start_date_picker, end_date_pck=end_date_picker, commitment_amount=commitment_amount
)

interactive(children=(DatetimePicker(value=Timestamp('2023-01-01 00:00:00+0000', tz='UTC'), description='Start…

<function __main__.render_choice(start_date_pck, end_date_pck, commitment_amount)>